<a href="https://colab.research.google.com/github/nhn09/NLP-notebooks/blob/main/darktriad_pt02_datacleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
nltk.download()
from gensim.models import Word2Vec
from nltk.corpus import stopwords
import re
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

stemmer = PorterStemmer()
wordnet = WordNetLemmatizer()

In [ ]:
import pandas as pd
import numpy as np


In [ ]:
dataset = pd.read_csv('/content/postwithScore.csv')

In [ ]:
dataset=dataset.drop(columns=['Unnamed: 0'])

In [ ]:
dataset.head()
dataset.columns

Index(['post ', 'Mach', 'LSRP12', 'LSRP2', 'NRSM'], dtype='object')

In [ ]:
dataset = dataset.drop(dataset[dataset['post '] =='undefined'].index).reset_index()

In [ ]:
finalDataSet = dataset.to_csv('finalDataset.csv')

In [ ]:
dataset.loc[dataset.Mach <51 , "Mach"] = 0
dataset.loc[dataset.Mach > 50, "Mach"] = 1


In [ ]:
dataset =  dataset[dataset['post '].notna()].reset_index()

In [ ]:
dataset['post '].shape

(8692,)

data cleaning with tfidf

In [ ]:
# sentences = nltk.sent_tokenize(paragraph)
corpus = []

for i in range(0,len(dataset)):
  sentences = nltk.sent_tokenize(text)
  review = re.sub('[^a-zA-Z]',' ',dataset['post '][i])
  review = review.lower()
  review = review.split()
  review = [wordnet.lemmatize(word) for word in review if not word in set(stopwords.words('english'))]
  review = ' '.join(review)
  corpus.append(review)

from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000)
X = cv.fit_transform(corpus).toarray()

In [ ]:
Y =dataset['Mach']

In [ ]:
Y.shape

(8692,)

In [ ]:
print(Y)

0       1.0
1       1.0
2       1.0
3       1.0
4       1.0
       ... 
8687    1.0
8688    1.0
8689    1.0
8690    1.0
8691    1.0
Name: Mach, Length: 8692, dtype: float64


In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size = 0.20 , random_state=42 )


# Naive bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB
machi_model = MultinomialNB().fit(X_train,Y_train)

y_pred = machi_model.predict(X_test)

from sklearn.metrics import mean_squared_error

mean_squared_error(Y_test,y_pred)


0.3881541115583669

In [ ]:
from sklearn import metrics

# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(Y_test, y_pred))

Accuracy: 0.6118458884416331


# SVM

In [ ]:
from sklearn import svm

#Create a svm Classifier
clf = svm.SVC(kernel='linear') # Linear Kernel

#Train the model using the training sets
clf.fit(X_train,Y_train)

#Predict the response for test dataset
y_pred_svm = clf.predict(X_test)

In [ ]:


# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(Y_test, y_pred_svm))

Accuracy: 0.6066705002875216
